In [1]:
import json

with open("/Users/badrinarayan/chatbot_django_langgraph/data/data.json", "r") as f:
    data = json.load(f)

data[0] 

{'url': 'https://dsphelp.syniti.com/802/solex/Home.htm',
 'title': 'SAP Advanced Data Migration by Syniti Help',
 'content': "SAP Advanced Data Migration by Syniti Help Ã— Can't find what you were looking for? Try these tips or SAP Advanced Data Migration by Syniti Help Migration Advanced Data Migration Console Target Design Construct Map Transform SQL AutoGen Migration Reports Data Quality ISA Stewardship Tier Assemble Automate Collect Common Integrate Promote System Administration Add-Ons Application Development Top Get in Touch Contact Support Access Product Ideas Portal Request Product Training Related Content Glossary Product Licenses Learn More **Access to pages and visibility of fields may be restricted based on the security settings of the current user. Copyright Â© 2022 BackOffice Associates, LLC d/b/a Syniti. and/or its affiliates. All rights reserved. Names appearing on the Site may be trademarks of their respective owners. Privacy Policy Trademarks Terms of Use"}

In [2]:
metadata = [
    {"url": item["url"], "title": item["title"]}
    for item in data
]

contents = [item["content"] for item in data]

metadata[:1], contents[:1]

([{'url': 'https://dsphelp.syniti.com/802/solex/Home.htm',
   'title': 'SAP Advanced Data Migration by Syniti Help'}],
 ["SAP Advanced Data Migration by Syniti Help Ã— Can't find what you were looking for? Try these tips or SAP Advanced Data Migration by Syniti Help Migration Advanced Data Migration Console Target Design Construct Map Transform SQL AutoGen Migration Reports Data Quality ISA Stewardship Tier Assemble Automate Collect Common Integrate Promote System Administration Add-Ons Application Development Top Get in Touch Contact Support Access Product Ideas Portal Request Product Training Related Content Glossary Product Licenses Learn More **Access to pages and visibility of fields may be restricted based on the security settings of the current user. Copyright Â© 2022 BackOffice Associates, LLC d/b/a Syniti. and/or its affiliates. All rights reserved. Names appearing on the Site may be trademarks of their respective owners. Privacy Policy Trademarks Terms of Use"])

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [4]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="rag_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [5]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_langchain_db")

In [6]:
from uuid import uuid4
from langchain_core.documents import Document

rag_documents = [
    Document(page_content=content, metadata=meta)
    for content, meta in zip(contents, metadata)
]

In [7]:
rag_documents

[Document(metadata={'url': 'https://dsphelp.syniti.com/802/solex/Home.htm', 'title': 'SAP Advanced Data Migration by Syniti Help'}, page_content="SAP Advanced Data Migration by Syniti Help Ã— Can't find what you were looking for? Try these tips or SAP Advanced Data Migration by Syniti Help Migration Advanced Data Migration Console Target Design Construct Map Transform SQL AutoGen Migration Reports Data Quality ISA Stewardship Tier Assemble Automate Collect Common Integrate Promote System Administration Add-Ons Application Development Top Get in Touch Contact Support Access Product Ideas Portal Request Product Training Related Content Glossary Product Licenses Learn More **Access to pages and visibility of fields may be restricted based on the security settings of the current user. Copyright Â© 2022 BackOffice Associates, LLC d/b/a Syniti. and/or its affiliates. All rights reserved. Names appearing on the Site may be trademarks of their respective owners. Privacy Policy Trademarks Terms

In [8]:
rag_ids = [str(uuid4()) for _ in rag_documents]

vector_store.add_documents(documents=rag_documents, ids=rag_ids)

['d016ab57-49a6-4efb-873c-f39e11221427',
 '30463de5-5d86-4ce0-844d-9a1a5ed47855',
 'c8376148-5378-4f77-8320-eca0a6d413d2',
 'f1d4f993-1ae5-4a9e-b431-e2732c102ce6',
 'cfbd1b1a-4329-4cb2-a15d-e5bd8843d914']

In [9]:
results = vector_store.similarity_search(
    "SAP Advanced Data Migration",
    k=2,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* SAP Advanced Data Migration by Syniti Help Ã— Can't find what you were looking for? Try these tips or SAP Advanced Data Migration by Syniti Help Migration Advanced Data Migration Console Target Design Construct Map Transform SQL AutoGen Migration Reports Data Quality ISA Stewardship Tier Assemble Automate Collect Common Integrate Promote System Administration Add-Ons Application Development Top Get in Touch Contact Support Access Product Ideas Portal Request Product Training Related Content Glossary Product Licenses Learn More **Access to pages and visibility of fields may be restricted based on the security settings of the current user. Copyright Â© 2022 BackOffice Associates, LLC d/b/a Syniti. and/or its affiliates. All rights reserved. Names appearing on the Site may be trademarks of their respective owners. Privacy Policy Trademarks Terms of Use [{'title': 'SAP Advanced Data Migration by Syniti Help', 'url': 'https://dsphelp.syniti.com/802/solex/Home.htm'}]
* ADM Overview ADM Adv

In [10]:
results = vector_store.similarity_search_with_score(
    "SAP Advanced Data Migration",
    k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.526971] SAP Advanced Data Migration by Syniti Help Ã— Can't find what you were looking for? Try these tips or SAP Advanced Data Migration by Syniti Help Migration Advanced Data Migration Console Target Design Construct Map Transform SQL AutoGen Migration Reports Data Quality ISA Stewardship Tier Assemble Automate Collect Common Integrate Promote System Administration Add-Ons Application Development Top Get in Touch Contact Support Access Product Ideas Portal Request Product Training Related Content Glossary Product Licenses Learn More **Access to pages and visibility of fields may be restricted based on the security settings of the current user. Copyright Â© 2022 BackOffice Associates, LLC d/b/a Syniti. and/or its affiliates. All rights reserved. Names appearing on the Site may be trademarks of their respective owners. Privacy Policy Trademarks Terms of Use [{'title': 'SAP Advanced Data Migration by Syniti Help', 'url': 'https://dsphelp.syniti.com/802/solex/Home.htm'}]


In [11]:
results = vector_store.similarity_search_by_vector(
    embedding=embeddings.embed_query("SAP Advanced Data Migration"), k=1
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* SAP Advanced Data Migration by Syniti Help Ã— Can't find what you were looking for? Try these tips or SAP Advanced Data Migration by Syniti Help Migration Advanced Data Migration Console Target Design Construct Map Transform SQL AutoGen Migration Reports Data Quality ISA Stewardship Tier Assemble Automate Collect Common Integrate Promote System Administration Add-Ons Application Development Top Get in Touch Contact Support Access Product Ideas Portal Request Product Training Related Content Glossary Product Licenses Learn More **Access to pages and visibility of fields may be restricted based on the security settings of the current user. Copyright Â© 2022 BackOffice Associates, LLC d/b/a Syniti. and/or its affiliates. All rights reserved. Names appearing on the Site may be trademarks of their respective owners. Privacy Policy Trademarks Terms of Use [{'url': 'https://dsphelp.syniti.com/802/solex/Home.htm', 'title': 'SAP Advanced Data Migration by Syniti Help'}]


In [12]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5}
)
retriever.invoke("SAP Advanced Data Migration")

[Document(id='d016ab57-49a6-4efb-873c-f39e11221427', metadata={'title': 'SAP Advanced Data Migration by Syniti Help', 'url': 'https://dsphelp.syniti.com/802/solex/Home.htm'}, page_content="SAP Advanced Data Migration by Syniti Help Ã— Can't find what you were looking for? Try these tips or SAP Advanced Data Migration by Syniti Help Migration Advanced Data Migration Console Target Design Construct Map Transform SQL AutoGen Migration Reports Data Quality ISA Stewardship Tier Assemble Automate Collect Common Integrate Promote System Administration Add-Ons Application Development Top Get in Touch Contact Support Access Product Ideas Portal Request Product Training Related Content Glossary Product Licenses Learn More **Access to pages and visibility of fields may be restricted based on the security settings of the current user. Copyright Â© 2022 BackOffice Associates, LLC d/b/a Syniti. and/or its affiliates. All rights reserved. Names appearing on the Site may be trademarks of their respect